In [4]:
!pip install reverse_geocoder

In [5]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import folium
from folium.plugins import HeatMap

from folium.plugins import FastMarkerCluster

from bs4 import BeautifulSoup

import reverse_geocoder as rg 
import pprint 
  

In [6]:
df = pd.read_json('data/data.json')


In [7]:
def fraud(string):
    if 'fraud' in string or 'lock' in string or 'tos_warn' in string:
        return True
    else:
        return False
df['label'] = df.acct_type.apply(lambda x: fraud(x))

In [8]:
df['description'] = df.description.apply(lambda x: BeautifulSoup(x).text)



#### Geographical visualization of all the vents

In [9]:
def generateBaseMap(default_location=[25.777471, -80.133433], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map


df_copy = df.copy()
df_copy['count'] = 1
base_map = generateBaseMap()
HeatMap(data=df_copy[['venue_latitude', 'venue_longitude', 'count']].groupby(['venue_latitude', 'venue_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)


base_map.save('images/events_heatmap.html')

In [10]:
df2 = df_copy[~df_copy['venue_latitude'].isnull()]
df2.shape

(13261, 46)

In [11]:
lat = df2['venue_latitude'].tolist()
lon = df2['venue_longitude'].tolist()
locations = list(zip(lat, lon))

map1 = folium.Map(location=[25.777471, -80.133433], zoom_start=11.5)
FastMarkerCluster(data=locations).add_to(map1)
map1.save('images/events_ClusteredMarker.html')

#### Geographical visualization of fraudulent events only

In [12]:
fraud = df2.query('label == 1')
fraud.shape

(1132, 46)

In [13]:

fraud['count'] = 1
base_map = generateBaseMap()
HeatMap(data=fraud[['venue_latitude', 'venue_longitude', 'count']].groupby(['venue_latitude', 'venue_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
base_map.save('images/fraud_heatmap.html')

/Users/franzmornau/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
lat = fraud['venue_latitude'].tolist()
lon = fraud['venue_longitude'].tolist()
locations = list(zip(lat, lon))

map2 = folium.Map(location=[25.777471, -80.133433], zoom_start=11.5)
FastMarkerCluster(data=locations).add_to(map2)
map2.save('images/fraud_ClusteredMarker.html')

#### Reverse Geocode Lookup

In [15]:
loc = df[['venue_latitude','venue_longitude','object_id']]
loc = loc[~loc['venue_latitude'].isnull()]
loc.head()

,venue_latitude,venue_longitude,object_id
0,25.777471,-80.133433,527017
1,32.776566,-79.930922,786878
2,33.944201,-118.080419,787337
4,42.353848,-71.044276,1114349
5,38.209797,-84.558831,1179983


In [16]:
results = []

for index, row in loc.iterrows():
    results.append(rg.search((row[0],row[1])))


Loading formatted geocoded file...


In [17]:
name = []
admin1 = []
admin2 = []
cc = []

for i in range(len(results)):
    
    name.append(results[i][0].get('name'))
    admin1.append(results[i][0].get('admin1'))
    admin2.append(results[i][0].get('admin2'))
    cc.append(results[i][0].get('cc'))
    

In [18]:
rg = pd.DataFrame(list(zip(name,admin1,admin2,cc)))
rg.columns = ['city_std','admin1','admin2','country_std']
rg.head()

,city_std,admin1,admin2,country_std
0,Miami Beach,Florida,Miami-Dade County,US
1,Charleston,South Carolina,Charleston County,US
2,Santa Fe Springs,California,Los Angeles County,US
3,Boston,Massachusetts,Suffolk County,US
4,Georgetown,Kentucky,Scott County,US


In [26]:
rg.to_pickle('data/rg.pkl')

In [19]:
loc2 = pd.concat([loc,rg],ignore_index = False,axis = 1,sort=False)

In [20]:
loc2.shape

(14267, 7)

In [22]:
loc2.to_pickle('data/loc2.pkl')

In [23]:
import pickle
pickle.dump(loc2, open('data/loc.p','wb'))

In [24]:
l3= pd.read_pickle('data/loc2.pkl')

In [25]:
l3.head()

,venue_latitude,venue_longitude,object_id,city_std,admin1,admin2,country_std
0,25.777471,-80.133433,527017.0,Miami Beach,Florida,Miami-Dade County,US
1,32.776566,-79.930922,786878.0,Charleston,South Carolina,Charleston County,US
2,33.944201,-118.080419,787337.0,Santa Fe Springs,California,Los Angeles County,US
3,NaN,NaN,NaN,Boston,Massachusetts,Suffolk County,US
4,42.353848,-71.044276,1114349.0,Georgetown,Kentucky,Scott County,US
